In [ ]:
# train_model_pytorch.ipynb

import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

# ----------------------
# CONFIG
# ----------------------
DATASET_PATH = "BrainDataset"
IMG_SIZE = 128
CATEGORIES = ["notumor", "glioma", "meningioma", "pituitary"]
NUM_CLASSES = len(CATEGORIES)
BATCH_SIZE = 32
EPOCHS = 15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------
# Custom Dataset
# ----------------------
class BrainTumorDataset(Dataset):
    def __init__(self, folder_path, categories, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform
        for idx, category in enumerate(categories):
            category_path = os.path.join(folder_path, category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                self.images.append(img_path)
                self.labels.append(idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        if self.transform:
            img = self.transform(img)
        return img, label

# ----------------------
# Transforms
# ----------------------
transform = transforms.Compose([
    transforms.ToTensor(),               # convert to [0,1] and add channel
])

# ----------------------
# Load datasets
# ----------------------
train_dataset = BrainTumorDataset(os.path.join(DATASET_PATH, "Training"), CATEGORIES, transform=transform)
test_dataset  = BrainTumorDataset(os.path.join(DATASET_PATH, "Testing"), CATEGORIES, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

# ----------------------
# Define CNN model
# ----------------------
class CNNModel(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128*14*14, 128)  # depends on IMG_SIZE=128
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNNModel().to(DEVICE)

# ----------------------
# Loss and optimizer
# ----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------------
# Training loop
# ----------------------
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / total
    train_acc = correct / total * 100
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")

# ----------------------
# Evaluate on test set
# ----------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = correct / total * 100
print(f"Test Accuracy: {test_acc:.2f}%")

# ----------------------
# Save the trained model
# ----------------------
torch.save(model.state_dict(), "brain_tumor_model_weights.pth")
print("Model saved as brain_tumor_model.pth")


Training samples: 5712, Testing samples: 1311
Epoch [1/15], Loss: 0.7429, Accuracy: 70.57%
Epoch [2/15], Loss: 0.4449, Accuracy: 82.58%
Epoch [3/15], Loss: 0.3463, Accuracy: 87.20%
Epoch [4/15], Loss: 0.2885, Accuracy: 89.32%
Epoch [5/15], Loss: 0.2247, Accuracy: 91.00%
Epoch [6/15], Loss: 0.1763, Accuracy: 93.73%
Epoch [7/15], Loss: 0.1552, Accuracy: 94.10%
Epoch [8/15], Loss: 0.1289, Accuracy: 95.31%
Epoch [9/15], Loss: 0.1056, Accuracy: 96.18%
Epoch [10/15], Loss: 0.0868, Accuracy: 96.95%
Epoch [11/15], Loss: 0.0753, Accuracy: 97.18%
Epoch [12/15], Loss: 0.0670, Accuracy: 97.57%
Epoch [13/15], Loss: 0.0589, Accuracy: 97.79%
Epoch [14/15], Loss: 0.0512, Accuracy: 98.20%
Epoch [15/15], Loss: 0.0521, Accuracy: 98.27%
Test Accuracy: 97.56%
Model saved as brain_tumor_model.pth


In [ ]:
# train_lung_cnn.py

import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ----------------------
# CONFIG
# ----------------------
DATASET_PATH = "LungDataset"  # main dataset folder containing train/test/valid
IMG_SIZE = 128
CATEGORIES = ["adenocarcinoma", "large cell carcinoma", "squamous cell carcinoma", "normal"]
NUM_CLASSES = len(CATEGORIES)
BATCH_SIZE = 32
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------
# DATASET CLASS
# ----------------------
class LungDataset(Dataset):
    def __init__(self, folder_path, categories, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform
        for idx, category in enumerate(categories):
            category_path = os.path.join(folder_path, category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                self.images.append(img_path)
                self.labels.append(idx)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.images[idx], cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# ----------------------
# TRANSFORMS
# ----------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ----------------------
# LOAD DATA
# ----------------------
train_dataset = LungDataset(os.path.join(DATASET_PATH, "train"), CATEGORIES, transform=transform)
test_dataset  = LungDataset(os.path.join(DATASET_PATH, "test"), CATEGORIES, transform=transform)
valid_dataset = LungDataset(os.path.join(DATASET_PATH, "valid"), CATEGORIES, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ----------------------
# CNN MODEL
# ----------------------
class CNNModel(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128*14*14, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNNModel().to(DEVICE)

# ----------------------
# LOSS & OPTIMIZER
# ----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------------
# TRAINING LOOP
# ----------------------
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = correct/total*100
    train_loss = running_loss/total
    
    # VALIDATION
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_acc = val_correct / val_total * 100
    val_loss /= val_total
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# ----------------------
# TEST EVALUATION
# ----------------------
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {test_correct/test_total*100:.2f}%")

# ----------------------
# SAVE MODEL
# ----------------------
torch.save(model.state_dict(), "lung_cnn_model.pth")
print("Saved model: lung_cnn_model.pth")


Epoch 1/20 | Train Loss: 1.1268, Train Acc: 47.31% | Val Loss: 1.0432, Val Acc: 37.50%
Epoch 2/20 | Train Loss: 0.8576, Train Acc: 60.36% | Val Loss: 1.3219, Val Acc: 45.83%
Epoch 3/20 | Train Loss: 0.8215, Train Acc: 63.95% | Val Loss: 1.0076, Val Acc: 63.89%
Epoch 4/20 | Train Loss: 0.6624, Train Acc: 72.92% | Val Loss: 0.8894, Val Acc: 69.44%
Epoch 5/20 | Train Loss: 0.5455, Train Acc: 77.98% | Val Loss: 0.8423, Val Acc: 61.11%
Epoch 6/20 | Train Loss: 0.5960, Train Acc: 75.86% | Val Loss: 0.6906, Val Acc: 66.67%
Epoch 7/20 | Train Loss: 0.4298, Train Acc: 82.54% | Val Loss: 0.8276, Val Acc: 65.28%
Epoch 8/20 | Train Loss: 0.3527, Train Acc: 85.97% | Val Loss: 0.7633, Val Acc: 73.61%
Epoch 9/20 | Train Loss: 0.2927, Train Acc: 89.07% | Val Loss: 0.6122, Val Acc: 73.61%
Epoch 10/20 | Train Loss: 0.1872, Train Acc: 93.15% | Val Loss: 0.7211, Val Acc: 75.00%
Epoch 11/20 | Train Loss: 0.2210, Train Acc: 91.19% | Val Loss: 0.6292, Val Acc: 77.78%
Epoch 12/20 | Train Loss: 0.1741, Train A